In [3]:
!pip install transformers

In [4]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV  #finding best model
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')


In [5]:
import pandas as pd

df=pd.read_csv('/content/draft_50000.csv')
df['text'] = df['text'].str.lower().str.replace('[^\w\s]', '', regex=True)
df.dropna(subset=['text', 'class'], inplace=True)
df.head()

,Unnamed: 0,text,class
0,51297,get crush guy definitely way old laugh literal...,0.0
1,24705,go to july 2018i hope 8 month enough change mind,1.0
2,185969,want live anymore 23 right thinking end life p...,1.0
3,201675,every time get period want kill already depres...,1.0
4,52701,story incomplete similar story op case dad spe...,1.0


In [6]:
df["class"].value_counts()

class
0.0    7899
1.0    7725
Name: count, dtype: int64

In [7]:
from sklearn import preprocessing
data = preprocessing.LabelEncoder()
df["text"] = data.fit_transform(df["text"])
df.head()

,Unnamed: 0,text,class
0,51297,4908,0.0
1,24705,5428,1.0
2,185969,14753,1.0
3,201675,3489,1.0
4,52701,12512,1.0


In [8]:
import nltk # Import the nltk module first
nltk.download('punkt')
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english') # Use words() instead of word()
stopwords.extend(['.',',','!','?',';',':'])
#Remove stopwords from a given text
def remove_stopword(txt):
  # Use word_tokenize instead of word_tokenizer
  tokens=nltk.word_tokenize(str(txt)) # Convert txt to string before tokenization
  new_tokens=[token for token in tokens if token.lower() not in stopwords]
  return ' '.join(new_tokens)
df['text']  =df['text'].apply(remove_stopword)
df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Unnamed: 0,text,class
0,51297,4908,0.0
1,24705,5428,1.0
2,185969,14753,1.0
3,201675,3489,1.0
4,52701,12512,1.0


In [9]:
# For BERT:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

## Want DistilBERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [10]:
tokenized = df["text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [11]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [12]:
np.array(padded).shape

(15624, 5)

In [13]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(15624, 5)

In [15]:
import torch

max_length = 512

# Assuming 'padded' is the NumPy array containing token IDs
input_ids = torch.tensor(padded)  # Convert padded to a PyTorch tensor
attention_mask = torch.tensor(attention_mask) # Convert to PyTorch tensor

# Truncate input_ids and attention_mask
input_ids = input_ids[:, :max_length]
attention_mask = attention_mask[:, :max_length]

# Proceed with the model inference
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [16]:
print(attention_mask.size())

torch.Size([15624, 5])


In [17]:
features = last_hidden_states[0][:,0,:].numpy()

In [21]:
c = df["text"]
#print(c)

0         4908
1         5428
2        14753
3         3489
4        12512
         ...  
15620    15374
15621     4461
15622    14653
15623      891
15624    12222
Name: text, Length: 15624, dtype: object


In [27]:
x_train, x_test, y_train, y_test = train_test_split(features,df["class"])

In [28]:

from sklearn.ensemble import RandomForestClassifier

model= RandomForestClassifier(n_estimators=100)

model.fit(x_train, y_train)


RandomForestClassifier()

In [29]:
import numpy as np
pred= model.predict(x_test)

from sklearn.metrics import classification_report
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         0.0       0.57      0.63      0.60      1945
         1.0       0.59      0.52      0.55      1961

    accuracy                           0.58      3906
   macro avg       0.58      0.58      0.58      3906
weighted avg       0.58      0.58      0.58      3906

